In [27]:
#from src.util import data_from_many
import os
from json import loads
import re
import datetime

publication = 'nydailynews'

datafiles = ['/datapool/news_articles/raw_data/'+publication+'/' + fname 
             for fname in os.listdir('/datapool/news_articles/raw_data/'+publication+'/') if '.json' in fname]

data = map(loads,data_from_many(datafiles))

In [28]:
nydailynews_date = re.compile('(?<=content=")\d{4}-\d{2}-\d{2}')
def get_nydailynews_date(html):
    match = nydailynews_date.search(html)
    if match:
        return datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
    else:
        return None
    
from collections import Counter
nydailynews_dates = Counter()

i = 0
for item in data:
    i+=1
    date = get_nydailynews_date(item['html'])
    if date:
        nydailynews_dates.update([date.isoformat()])
    if i % 1000 == 0:
        print(i,end = '\r')

In [3]:
with open(publication+'_datelist.csv','w') as file:
    for item in sorted(list(nydailynews_dates.items()),key = lambda x: -int(x[0].split('-')[0])):
        file.write('%s, %i\n' % item)

In [26]:
from itertools import chain

def get_data(filename):
    with open(filename,'r') as file:
        for line in file:
            yield line
            
def get_data_from_many(filenames):
    gens = chain.from_iterable([get_data(filename) for filename in filenames])
    for line in gens:
        yield line
    